# Monthly Retail Trade Survey ETL and Analysis

Melissa Belfer

# Index

- [Abstract](#Abstract)
- [1. Introduction](#1.-Introduction)
- [2. Extract-Transform-Load](#2.-Extract-Transform-Load)
    - [2.1 The ETL Process](#2.1-The-ETL-Process)
    - [2.2 Data Exploration](#2.2-Data-Exploration)
    - [2.3 Data Preparation](#2.3-Data-Preparation)
    - [2.4 Read the Data Using Python](#2.4-Reading-the-Data-Using-Python)
         - [2.4.1 Reading Sample Data](#2.4.1-Reading-Sample-Data)
         - [2.4.2 Reading the MRST Data](#2.4.2-Reading-the-MRST-Data)
    - [2.5 Writing an Installation Script](#2.5-Writing-an-Installation-Script)
- [3. Analysis and Visualization](#3.-Project-Description)
    - [3.1 Running Queries in MySQL Workbech](#3.1-Running-Queries-in-MySQL-Workbech)
    - [3.2 Running Queries From Python](#3.2-Running-Queries-From-Python)
    - [3.3 Explore Trends](#3.3-Explore-Trends)
    - [3.4 Explore Percentage Change](#3.4-Explore-Percentage-Change)
    - [3.5 Explore Rolling Time Windows](#3.5-Explore-Rolling-Time-Windows)
- [Conclusion](#Conclusion)
- [References](#References)

[Back to top](#Index)


##  Abstract


After creating an automated ETL process on the Monthly Retail Trader Survey data, I am able to analyze the dataset.  There is an uptrend in the retail and food service businesses as well as the beer, wine, and liquor stores.  There is a common decrease in spending across businesses during March-April 2020, which is when covid lockdown happened.  There is also another common decrease in spending in 2008-2009 during the Great Recession.  There is a seasonal trend yearly of an increase in spending during November and December which is during the holiday season. 

[Back to top](#Index)


## 1. Introduction


I will be extracting data from the Monthly Retail Trade Survey since 1992 to the beginning of 2021 and performing an ETL on the data, analyzing the time series, and representing the trends through visual representation.  First, I start by analyzing the Excel sheet that the MRTS data is in and notice the common patterns.  I use the common patterns to build an automated ETL process in Python that loads the data into a table in a MySQL database. I extract every sheet from the excel, transform the dataset by transposing the dataset, and combining all the cleaned datasets into one DataFrame.  This final DataFrame is then loaded into a table in MySQL.  I then begin performing my analysis by finding trends in the different kind of businesses over the years.  This involves querying the dataset to produce a subset of the main data and plotting the data on a graph. After plotting the data, I can notice the trends in the dataset.  I can determine what trends have an uptrend, downtrend, or just stationary.  I look at trends with the raw values, the rolling averages, and the percentage change.  I am able to look at a less volatile dataset with rolling averages versus the raw values.  I am also able to look at the percentage change that shows more unusual changes in the dataset. 

[Back to top](#Index)

## 2. Extract-Transform-Load


[Back to top](#Index)

### 2.1 The ETL Process


In order to perform ETL on the MRTS dataset, I need to eliminate any unnecessary information from the excel sheet which can be done manually or using pandas.  I then need to load each sheet from the excel file into a python program and transform the data by cleaning up the datasets to only include necessary information.  This process involves removing any unnecessary rows or columns, cleaning up column names, changing datatypes if necessary, and deriving new columns from the source columns if necessary.  After the dataset is cleaned and transformed into the format necessary, I load it into a database.  For this situation, I load the cleaned and transformed data into a MySQL database.

[Back to top](#Index)

### 2.2 Data Exploration


The monthly retail trade survey is used to capture the monthly sales for each kind of business.  It contains the monthly sales for grocerys stores, furniture stores, department stores, and a lot more.  It will then sum all of the data from each kind of business for the year and capture the total amount for that kind of business. 

[Back to top](#Index)

### 2.3 Data Preparation




I  decided to modify the excel sheet by importing each sheet into python using the pandas read_excel() function.  This function allowed me to skip the first 4 rows and the bottom 50 rows from each excel sheet since I only want the not adjusted data from the excel sheet. The only values I wanted to keep were the date columns and the data under ‘Kind of Business’. Once I imported the data into a DataFrame, I dropped the first column and the total column.  

When analyzing time series data, the date values are in a column next to the value column you are analyzing.  Although in the excel sheet the date values were the column headers, so I decided to un-pivot the data by using the pandas melt() function to pivot all the date columns but not the ‘Kind of Business’ column.  After I transposed the DataFrame, I changed all values that were (S) or (NA) to zero because I wanted only numeric values for my analysis. I also decided to drop all rows containing a null value since that data won’t be of use during the analysis phase.  The column that contains the date field is an object data type, but I want the column to be of a date datatype.  So I converted the column to a date datatype using the pandas function to_datetime().  I then concatenated all the DataFrames from each sheet in the excel file before loading the data into the database. 


[Back to top](#Index)

### 2.4 Read the Data Using Python


The Pandas library contains a function **read_excel()** that can read each sheet of an excel into a DataFrame.  The function takes the arguments of filename, sheet_name, skiprows, and skipfooter.  The skiprows argument allows to skip the top rows of the excel sheet, and the skipfooter argument allows me to skip the bottom rows of the excel sheet that I don't want to import into my DataFrame.

By using the read_excel() function, I did not need to export each excel sheet to a csv.  The process I did to read sample data is different then the process I took to read the mrts data. The code for reading sample data from a csv with python is below.  The code I used in my installation script is in the files **mrts_etl.py** and **etl.py**. The main script is **mrts_etl.py** and it imports **etl.py** functions.

[Back to top](#Index)

### 2.4.1 Reading Sample Data


The sample dataset I used to test my Python script is a csv that covers the unique case scenarios in the MRTS dataset.  The columns are 'Unnamed: 1', 'Jan. 2020','Feb. 2020', and 'total'. The values in 'Unnamed: 1' contains string and the column will be converted to column name 'KindOfBusiness'.  The values in the columns 'Jan. 2020', 'Feb. 2020', and 'total' contain either a number with a comma in it, a string of (S), a string of (NA), or it is empty. 

I used pandas to read the csv into a dataframe by using the function *pd.read_csv()*.  

In [4]:
import pandas as pd

df = pd.read_csv('sample_data.csv')
print(df)

            Unnamed: 1 Jan. 2020 Feb. 2020    total
0     Furniture stores       NaN     5,691  100,691
1       Grocery stores    64,785    59,076  105,691
2   Electronics stores     5,435       (S)  150,691
3  Retail sales, total      (NA)     5,435  150,691
4      New car dealers    72,074    82,174  160,381
5     Used car dealers     7,074    10,174   30,381


I then preceded to clean the dataframe by dropping the total column, renaming the column 'Unnamed: 1' to 'KindOfBusiness', transposing the dataframe, replacing all values of (S) & (NA) with 0, dropping all rows containing a null value, and converting the value function to a datatype of date.

In [6]:
# Drop last column
df.drop(df.columns[3], axis=1, inplace=True)

# Rename first column
colnames = {'Unnamed: 1': 'KindOfBusiness'}
df.rename(columns=colnames, inplace=True)
df

,KindOfBusiness,Jan. 2020,Feb. 2020
0,Furniture stores,NaN,"5,691"
1,Grocery stores,"64,785","59,076"
2,Electronics stores,"5,435",(S)
3,"Retail sales, total",(NA),"5,435"
4,New car dealers,"72,074","82,174"
5,Used car dealers,"7,074","10,174"


In [8]:
# Unpivot dataframe
df = pd.melt(df, id_vars='KindOfBusiness', value_vars=df.columns[1:])
df

,KindOfBusiness,variable,value
0,Furniture stores,Jan. 2020,NaN
1,Grocery stores,Jan. 2020,"64,785"
2,Electronics stores,Jan. 2020,"5,435"
3,"Retail sales, total",Jan. 2020,(NA)
4,New car dealers,Jan. 2020,"72,074"
5,Used car dealers,Jan. 2020,"7,074"
6,Furniture stores,Feb. 2020,"5,691"
7,Grocery stores,Feb. 2020,"59,076"
8,Electronics stores,Feb. 2020,(S)
9,"Retail sales, total",Feb. 2020,"5,435"


In [9]:
# Replace all the (S) & (NA) to 0
df.replace('(S)', 0, inplace=True)
df.replace('(NA)', 0, inplace=True)

# Drop all rows where there is a null value
df.dropna(axis=0, inplace=True)

# Convert variable column to date format
df['Date'] = pd.to_datetime(df['variable'])
df.drop(columns=['variable'], inplace=True)
df

,KindOfBusiness,value,Date
1,Grocery stores,"64,785",2020-01-01
2,Electronics stores,"5,435",2020-01-01
3,"Retail sales, total",0,2020-01-01
4,New car dealers,"72,074",2020-01-01
5,Used car dealers,"7,074",2020-01-01
6,Furniture stores,"5,691",2020-02-01
7,Grocery stores,"59,076",2020-02-01
8,Electronics stores,0,2020-02-01
9,"Retail sales, total","5,435",2020-02-01
10,New car dealers,"82,174",2020-02-01


[Back to top](#Index)

### 2.4.2 Reading the MRTS Data




In order to read the mrts dataset into Python, I create a list of all the sheet names but exclude sheet 2021.  All of the sheets except 2021 have a similar format. I first read the sheet into a dataframe using pandas read_excel() function and include the rows I want to skip at the beginning of the sheet and the rows at the bottom I want to skip.  I decide to skip the first 4 rows and the bottom 50.

For the 2021 sheet, I drop the first column and last two columns.  I then rename the columns 'Unnamed: 1' to 'KindOfBusiness' and 'Feb. 2021(p)' to 'Feb. 2021'.  After renaming the column, I transpose the dataframe by using the melt() function in pandas.  I transpose all the date columns.  After the dataframe is unpivoted, I clean up the dataframe by changing (S) and (NA) values to 0, drop all rows containing nulls as this data is no longer useful for analysis, and create a date column with the datatype datetime64[ns] for the Months and Years.  

I set the 2021 cleaned dataframe as the final dataframe.  I then loop through all the rest of the sheets and concat the cleaned dataframes to the 2021 dataframe.  The only steps that differ from the steps performed on the 2021 sheet is that I just drop the first and last column, and rename only the column 'Unnamed: 1' to 'KindOfBusiness'. The rest of the actions of cleaning the data remain the same. 

Functions below are used to clean and transform my data as well as reduce redundancy in my code. 

In [ ]:
def unpivot_dataframe(df):
    """ Unpivots the source dataframe for each excel sheet """
    return pd.melt(df, id_vars='KindOfBusiness', value_vars=df.columns[1:])

def replace_with_zero(df, str):
    """ Replaces string value with 0 in the DataFrame """
    df.replace(str, 0, inplace=True)
    return df

def clean_source_data(df):
    """ Cleans source data for (S), (NA), null values, and column data types/renaming """
    # Replace all the (S) & (NA) to 0
    df = replace_with_zero(df, '(S)')
    df = replace_with_zero(df, '(NA)')

    # Drop all rows where there is a null value
    df.dropna(axis=0, inplace=True)

    # Convert variable column to date format
    df['Date'] = pd.to_datetime(df['variable'])
    df.drop(columns=['variable'], inplace=True)

    df.rename(columns={'value': 'Value'}, inplace=True)
    return df

The code below is in the main script mrts_etl.py which is part of my installation script.  This extracts all the sheets from the excel file, cleans them, and transforms the data into what will be the best format for analysis. 

In [ ]:
# Source Data Details
filename = 'mrtssales92-present.xls'
sheets = ['2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1994', '1993', '1992']

In [ ]:
# Need to clean up sheet 2021 separately as it does not match other sheets
finaldf = pd.read_excel(filename, sheet_name='2021', skiprows=4, skipfooter=50)
finaldf.drop(finaldf.columns[[0,4,5]], axis=1, inplace=True)
# Rename columns
colnames = {'Unnamed: 1': 'KindOfBusiness', 'Feb. 2021(p)': 'Feb. 2021'}
finaldf.rename(columns=colnames, inplace=True)
# Unpivot the dataframe
finaldf = unpivot_dataframe(finaldf)
# Clean up the rows by changing (S) & (NA) to 0, dropping null rows, and creating date column
finaldf = clean_source_data(finaldf)

for sheet in sheets:
    # Skip first 4 rows and skip last 50 rows
    df = pd.read_excel(filename, sheet_name=sheet, skiprows=4, skipfooter=50)
    # Drop first column and last column
    df.drop(df.columns[[0,14]], axis=1, inplace=True)
    # Rename first column
    colnames = {'Unnamed: 1': 'KindOfBusiness'}
    df.rename(columns=colnames, inplace=True)
    # Unpivot the dataframe
    df = unpivot_dataframe(df)
    # Clean up the rows by changing (S) & (NA) to 0, dropping null rows, and creating date column
    df = clean_source_data(df)
    # Merge the data into one dataframe
    finaldf = pd.concat([finaldf,df], ignore_index=True)

![MRTS ETL Output](ETL_output.png)

[Back to top](#Index)

### 2.5 Writing an Installation Script

The details above describe the extraction, cleaning, and transforming steps for mrts data.  The only step that was added to complete my Python installation script is loading the DataFrame into a table in MySQL.  In order to do this, I create a yaml file to contain my database properties and use those properties to connect to mysql using the mysql.connector library.  I then use the connection to create a cursor object that I use to execute my queries.  I create an mrts database and then insert each row in the final DataFrame into the mrts table. I then commit my changes and close my cursor and database connection.

The main script is **mrts_etl.py**.  The other script that contains all the ETL functions I created to make my code reusable is in the file **etl.py**, which I then imported those functions in my main script. 

My property file is **db.yaml**.  I did not include the file in my submission as it includes a password.  Example of file excluding the password is below. 

```yaml
host: "localhost"
user: "root"
pass: ""
db: "mrts"
```

My source file that contains the data is **mrtssales92-present.xls**.

#### The functions below are used in my installation script to connect to my database, create a database, create a table, insert data, and disconnect my database.

In [ ]:
import pandas as pd
import mysql.connector

def connect_db(db):
    """ Connect to the MySQL Database """
    config = {
        'user': db['user'],
        'password': db['pass'],
        'host': db['host'],
        'database': db['db'],
        'auth_plugin': 'mysql_native_password'
    }
    cnx = mysql.connector.connect(**config)
    return cnx

def disconnect_db(cursor, cnx):
    """ Disconnect from the MySQL Database """
    cursor.close()
    cnx.close()

In [ ]:
def create_db(cursor):
    """ Create DataBase mrts """
    drop_db = 'DROP DATABASE IF EXISTS `mrts`'
    cursor.execute(drop_db)
    create_db = 'CREATE DATABASE IF NOT EXISTS `mrts`'
    cursor.execute(create_db)
    use_db = 'USE `mrts`'
    cursor.execute(use_db)
    cursor.execute('SET NAMES UTF8MB4;')
    cursor.execute('SET character_set_client = UTF8MB4;')

In [ ]:
def create_table(cursor):
    """ Create mrts Table """
    query = (f"CREATE TABLE `mrts` (`KindOfBusiness` varchar(100) NOT NULL, `Value` float(10,2) NOT NULL, `Date` DATE NOT NULL, PRIMARY KEY (KindOfBusiness, Date))")
    cursor.execute(query)

In [ ]:
def insert_data(cursor, df):
    """ Insert DataFrame into mrts table """
    for index, row in df.iterrows():
        cursor.execute('INSERT INTO mrts (KindOfBusiness,Value,Date) VALUES(%s, %s, %s);', (row[0], row[1], str(row[2])))



The output below shows that my python installation script was able to load data into the database named 'mrts' and load all the sheets data into one table called 'mrts'.

![Output of Installation Script](installation_script_output.png)


[Back to top](#Index)

## 3. Analysis and Visualization



The benefit of running queries in a Python environment is that I can use matplotlib and other graphing libraries to visualize the dataset.  MySQL workbench is very nice workspace for writing queries and looking at the values, but it doesn't have the potential for data visualization like Python does.  

[Back to top](#Index)

### 3.1 Running Queries in MySQL Workbench


I used the below queries to verify that the data was loaded into the table correctly. 

```sql
USE mrts;
SELECT * FROM mrts;
SELECT COUNT(*) FROM mrts; -- returns 21370
SELECT COUNT(DISTINCT KindOfBusiness) FROM mrts; -- returns 65
SELECT KindOfBusiness, COUNT(*) FROM mrts GROUP BY KindOfBusiness;
```

Before I loaded the data into MySQL, I received the count of the total rows which was 21,370.  This count of the rows matches the count of the rows I receive when I run this query in MySQL Workbench.

I verify that all of the categorys are loaded into the table by getting the count of the distinct kind of businesses.  I then look at the counts of how many records are loaded into the table for each kind of business. 

![Data Verification](data_verification.png)

[Back to top](#Index)

### 3.2 Running Queries From Python



I tested the same queries in the Python environment.  Below shows the code written in python, and the results from the queries.

In [ ]:
def test_queries(cursor):
    cursor.execute('SELECT * FROM mrts;')
    # print all the rows
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute('SELECT COUNT(*) FROM mrts;')
    print(cursor.fetchone())
    
    cursor.execute('SELECT COUNT(DISTINCT KindOfBusiness) FROM mrts')
    print(cursor.fetchone())

    cursor.execute('SELECT KindOfBusiness, COUNT(*) FROM mrts GROUP BY KindOfBusiness;')
    # print all the rows
    for row in cursor.fetchall():
        print(row)

Shows the results of the query in python for 
```sql
SELECT * FROM mrts;
```
![Select All](select_all_query_result.png)

Shows the results of the queries in python for 
```sql
SELECT COUNT(*) FROM mrts; -- first row
SELECT COUNT(DISTINCT KindOfBusiness) FROM mrts; -- second row
SELECT KindOfBusiness, COUNT(*) FROM mrts GROUP BY KindOfBusiness; -- rest of the rows
```

![Other Query Results](python_query_results.png)

[Back to top](#Index)

### 3.3 Explore Trends


I created the below function to reduce the redundancy of my code when plotting graphs.

In [ ]:
import matplotlib.pyplot as plt
def plot_data(x, y, title, xlabel='Date', ylabel='Value'):
    plt.plot(x,y)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

#### Economic Trend

An economic trend is a trend in the economy that can grow upwards, downwards, or stationary.  If the trend is stationary, it doesn't show any slope which means the economy is neither growing or shrinking.

It is important measure to predict quantities like spending patterns because it shows if people are spending a lot, then the economy is growing and wages are also growing.  Unlike if people are not spending, then the economy is shrinking and unemployment is on the rise. 

#### Trend for the Retail and Food Services Category

There is an uptrend for the total sales for the retail and food services categories.  As displayed in the graph below. 

In order to see the data displayed clearly, I sorted the dates in ascending order. 

![Retail and Food Services Category](total_sales_retail_food_trend.png)


In [ ]:
df_retail_food_total = df[df['KindOfBusiness'] == 'Retail and food services sales, total'].copy()
# Sort data in ascending order
df_retail_food_total.sort_values(by=['Date'], inplace=True)
plot_data(df_retail_food_total['Date'], df_retail_food_total['Value'], 'Total Sales for the Retail and Food Services From 1992-2021')


#### Comparison between bookstores, sporting goods stores, and hobbies, toys, and games stores businesses

When comparing the different businesses, sporting goods stores had the highest upward trend from 1992 to 2021.  It also has the highest values compared to the other stores.  Sporting good stores grew the fastest especially from 2008 to the present. During the time when sporting good stores started growing faster, bookstores started going downwards.  

There was a drop in sales in the year 2020 during the month of April, which makes sense since covid-19 lockdown happened in March 2020. 


![Comparison](retail_business.png)

Below shows a zommed version of the above graph, which shows the data from 1997 to 1999.  In the below graph we can see a seasonal pattern.  All three retail businesses spike between the months of November and December, which makes sense as that is the holiday season.  Another interesting season spike is with book stores during the summer months.  It is not as huge of a spike as during the holiday season, but it still increases during the months of July and August.  

As it is hard to see the seasonal patterns in a yearly graph, I show the same data but monthly for the year of 1998. 

A yearly graph is good to get a general view of the trend, but to see a closer view of seasonal spikes it is better to have a monthly view.  

Below shows the code used to produce a yearly view vs a monthly view of the comparison

In [ ]:
# Comparison between bookstores, sporting goods stores, and hobbies, toys, and games stores businesses
df_sporting = df.loc[(df['KindOfBusiness'] == 'Sporting goods stores')].sort_values(by=['Date']).copy()
df_hobbies = df.loc[df['KindOfBusiness'] == 'Hobby, toy, and game stores'].sort_values(by=['Date']).copy()
df_book = df.loc[df['KindOfBusiness'] == 'Book stores'].sort_values(by=['Date']).copy()

plt.plot(df_sporting['Date'],df_sporting['Value'], label='Sporting goods stores')
plt.plot(df_hobbies['Date'],df_hobbies['Value'], label='Hobby, toy, and game stores')
plt.plot(df_book['Date'],df_book['Value'], label='Book stores')
plt.legend()
plt.title('Comparison of retail businesses: bookstores, sporting goods stores, and hobbies, toys, and games stores')
plt.xlabel('Date')
plt.ylabel('Value')
plt.show()

df = df[(df['Date'] > '1997-12-01') & (df['Date'] < '1999-01-01')]
df_sporting = df.loc[(df['KindOfBusiness'] == 'Sporting goods stores')].sort_values(by=['Date']).copy()
df_hobbies = df.loc[df['KindOfBusiness'] == 'Hobby, toy, and game stores'].sort_values(by=['Date']).copy()
df_book = df.loc[df['KindOfBusiness'] == 'Book stores'].sort_values(by=['Date']).copy()

plt.plot(df_sporting['Date'],df_sporting['Value'], label='Sporting goods stores')
plt.plot(df_hobbies['Date'],df_hobbies['Value'], label='Hobby, toy, and game stores')
plt.plot(df_book['Date'],df_book['Value'], label='Book stores')
plt.legend()
plt.title('Comparison of retail businesses: bookstores, sporting goods stores, and hobbies, toys, and games stores')
plt.xlabel('Date')
plt.ylabel('Value')
plt.show()

![Seasonal](seasonal_retail_trend.png)

![Monthly 1998](retail_1998_monthly.png)

[Back to top](#Index)

### 3.4 Explore Percentage Change


The percentage change shows what percentage of a value has changed over a period of time.  This is used in stocks to show the percentage change at the end of the day of the stock prices.  The percentage change is important because it can help economists predict and estimate the next rise or fall.  The percent change is unitless which helps us avoid looking at the units of something since that can be misleading.  


#### Women's vs Men's Clothing

After calculating the percent change of women's and men's clothing spending from 1992 to 2021, it shows that the percent change for both men and women's clothing stays very conistent.  These two businesses both involve buying clothes, but for different genders.  The percent change over the years stays stationary except during the year 2020 when we see a fluctuation in the percent change. The percent change stays under 1% except during the year 2020 when the women's clothing percent change jumps above 1%.  
There is a consistent seasonal percentage change jump during the months from Novemeber to January where spending goes up during the holidays and decreases fast when the holidays are over. 

In [ ]:
def percent_change(df):
    """ Calculates the percent change for women's and mens clothing """
    df_women = df.loc[(df['KindOfBusiness'] == "Women's clothing stores")].sort_values(by=['Date']).set_index('Date').drop(columns='KindOfBusiness').copy()
    df_men = df.loc[df['KindOfBusiness'] == "Men's clothing stores"].sort_values(by=['Date']).set_index('Date').drop(columns='KindOfBusiness').copy()

    df_women_percent = df_women.pct_change()
    df_men_percent = df_men.pct_change()

    plt.plot(df_women_percent.index,df_women_percent['Value'], label='Women')
    plt.plot(df_men_percent.index,df_men_percent['Value'], label='Men')
    plt.legend()
    plt.title("Comparison of women's clothing and men's clothing businesses")
    plt.xlabel('Date')
    plt.ylabel('Percent Change')
    plt.show()


![Comparison](clothing.png)
![Closer Look](percent_2020.png)

[Back to top](#Index)

### 3.5 Explore Rolling Time Windows


The rolling time window lets you perform operations on a subset of data for a period of time, but instead of just receiving the average amount for a time period of January 1st to December 31st.  It allows you to compute the average amount for the last year.  It doesn't matter that the day is not December 31st. If the day is July 15th, it will still return the average amount for the last year.  This is really important when you want to consistently get the average spending of data for the last week or month. 

Rolling time windows are important when showing an economic trend since it's used to smooth the data which can help notice changes in the economy versus a non-smoothed peak.  This can reduce the volatility of the data trends.  Economists will use rolling time windows to smooth the data to analyze the trends better.

#### Gas Station Sales 

The gas station sales actual value versus rolling average of 3 and 6 months are very similar.  The only difference is the peaks on the graph are slightly smaller. 

In the graph we can also see a drastic drop in gas sales from 2008 to 2009.  This makes sense since the great recession was during this time period. Another drastic drop in gas sales happened from end of 2019 to beginning of 2020, which is during the time covid-19 lock-down happened. 


![Gas Prices](rolling_avg_gas.png)

In [ ]:
# Code that produced the graph above
df_gas= df.loc[(df['KindOfBusiness'] == 'Gasoline stations')].sort_values(by=['Date']).copy()

df_gas['RA_3'] = df_gas["Value"].rolling(3, win_type ='triang').mean()
df_gas['RA_6'] = df_gas["Value"].rolling(6, win_type ='triang').mean()
print(df_gas)

plt.plot(df_gas['Date'],df_gas['Value'], label='Value')
plt.plot(df_gas['Date'],df_gas['RA_3'], label='Rolling Avg 3 Months')
plt.plot(df_gas['Date'],df_gas['RA_6'], label='Rolling Avg 6 Months')
plt.legend()
plt.title("Rolling Average of Gas Prices over 3 and 6 months")
plt.xlabel('Date')
plt.ylabel('Value')
plt.show()

#### Beer, wine, and liquor stores

I compared the value, rolling average over 3 months, and rolling average over 6 months for the sales at beer, wine, and liquor stores. 

The graph shows an upward trend from 1992 to present of the sales at beer, wine, and liquor stores.  The average sales shows less volatile spending peaks than the actual sales per month.  

When looking at the rolling averages versus the value zoomed in on the year of 2017, we can see the seasonal trends over the holidays are not as drastic with the average.  Even though there is a seasonal trend of sales increased at beer, wine, and liquor stores during the holidays, the average sales are still increasing over the years.  The average over several months of sales helps reduce the volatility of the data. 

![Alcohol Stores](rolling_avg_alc.png)
![Alcohol Stores](roll_alc_2017png.png)

In [ ]:
# Code used to produce graph above
df_alochol= df.loc[(df['KindOfBusiness'] == 'Beer, wine, and liquor stores')].sort_values(by=['Date']).copy()

df_alochol['RA_3'] = df_alochol["Value"].rolling(3, win_type ='triang').mean()
df_alochol['RA_6'] = df_alochol["Value"].rolling(6, win_type ='triang').mean()

plt.plot(df_alochol['Date'],df_alochol['Value'], label='Value')
plt.plot(df_alochol['Date'],df_alochol['RA_3'], label='Rolling Avg 3 Months')
plt.plot(df_alochol['Date'],df_alochol['RA_6'], label='Rolling Avg 6 Months')
plt.legend()
plt.title("Rolling Average of Beer, wine, and liquor stores over 3 and 6 months")
plt.xlabel('Date')
plt.ylabel('Value')
plt.show()

[Back to top](#Index)

## Conclusion




Out of the businesses considered, book stores sales have been going down over the last few years.  This seems like it will attract the least amount of spending.  The retail and food service businesses seem most likely to attract the most spending.  There is a consistent uptrend over the years since 1992 of sales.  


[Back to top](#Index
)
## References



* "About the Monthly Retail Trade Survey." United States Census Bureau. January 16, 2009. 
June 1, 2022. https://www.census.gov/retail/mrts/about_the_surveys.html 
* Admin. "Why Do Economists Use Percentage Change To Calculate Elasticity Of Demand." Micro B Life. March 19, 2022. June 1, 2022. https://www.microblife.in/why-do-economists-use-percentage-change-to-calculate-elasticity-of-demand/
* Howland, Daphne. "Monthly retail sales from the US Commerce Department." Retail Dive.  Industry Dive, March 17, 2020. June 1, 2022. https://www.retaildive.com/news/monthly-retail-sales-from-the-us-commerce-department/574252/
* "Smoothing Data with Moving Averages." Federal Reserve Bank of Dallas. June 1, 2022. https://www.dallasfed.org/research/basics/moving.aspx
* "What is an Economic Trend?" bartleby. Barnes & Noble Education, Inc., 
April 19, 2022. June 1, 2022. https://www.bartleby.com/subject/business/concepts/economic-trends
